<a href="https://colab.research.google.com/github/anandraiyer/access_forums_eval/blob/main/testing_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/content/drive/MyDrive/final.csv')

In [3]:
data.columns

Index(['Thread', 'DateTime', 'Author', 'Post', 'ParentPosts', 'PostID',
       'ThreadID', 'AuthorID', 'OriginID', 'DialogAct', 'ParentID_List'],
      dtype='object')

In [4]:
df = data[['Thread','ThreadID','PostID','DateTime','Author','Post','ParentID_List']]

In [5]:
df.head()

,Thread,ThreadID,PostID,DateTime,Author,Post,ParentID_List
0,Testing the new Site!,1,0,2015-11-30 11:59:19.544597,Tim Ford,Thank you very much for all those who worked o...,-1
1,Netflix not accessible to blind people using a...,2,1,2015-11-30 12:05:11.019288,Tim Ford,"Hi All, For those out there who want to use N...",-1
2,Testing the new Site!,1,2,2015-11-30 12:06:01.976409,"Walker, Michael E","Hi Tim, the group is working fine. I got your ...",[0]
3,Testing the new Site!,1,3,2015-11-30 12:12:07.800324,ken lawrence,Should the JDH mail be deleted?,[0]
4,Netflix not accessible to blind people using a...,2,4,2015-11-30 12:14:27.873186,Greg Nickel,Will do…,[1]


In [67]:
for i in df[df['ThreadID']==34]['Post']:
  print(i)

Hi guys, I am licensed for jaws 17 but had to down grade back to 16 due to an issue. When I read an email message with outlook express,  jaws says blank and I keep having to alt f 4 out of it. Have anyone else had this problem? I use windows 10 pro 64 bit.  Maheen
I don't know for sure, but I would guess that this is because Jaws   17 no longer supports Outlook express, which hasn't been supported   by Microsoft for some time.
If JAWS 17 no longer supports Outlook Express, what is the better provider to use? Kevin
Hi Kevin:I personally use Outlook 2013 on a laptop and just recently upgraded to Outlook 2016 on my desktop. I am also using Windows 10. Until just recently I was using Windows 8. 1 with outlook 2013. Although some have expressed a dislike for it, I do have clients who use windows live mail with JAWS and seem to like it.   Jeanette McAllister PhD President / CEOAssistive Technology Tutor                 A Heart Staffing                          PO Box 1277Franklin, VA 23851Mo

In [68]:
from itertools import chain
from itertools import product
from itertools import starmap
from functools import partial
import networkx as nx

In [69]:
def get_conversation_dag(tid):
  temp = df[df['ThreadID']==tid]

  threads = list(temp['Thread'])
  postids = list(temp['PostID'])
  posts = list(temp['Post'])
  datetimes = list(temp['DateTime'])
  authors = list(temp['Author'])
  parentids = list(temp['ParentID_List'])

  G = nx.DiGraph()
  G.add_nodes_from(posts)

  edges = []
  for i in range(len(postids)):
    parent = parentids[i]
    child = postids[i]

    if parent == "-1":
      continue
    else:
      parent = int(parent[1:-1])
    edges.append((parent,child))
  G.add_edges_from(edges)
  return G

In [70]:
def get_subthreads(tid):
  temp = df[df['ThreadID']==tid]

  threads = list(temp['Thread'])
  postids = list(temp['PostID'])
  posts = list(temp['Post'])
  datetimes = list(temp['DateTime'])
  authors = list(temp['Author'])
  parentids = list(temp['ParentID_List'])

  G = nx.DiGraph()
  G.add_nodes_from(posts)

  edges = []
  for i in range(len(postids)):
    parent = parentids[i]
    child = postids[i]

    if parent == "-1":
      continue
    else:
      parent = int(parent[1:-1])
    edges.append((parent,child))
  G.add_edges_from(edges)
  chaini = chain.from_iterable
  roots = (n for n,d in G.in_degree() if d==0)
  leaves = (n for n,d in G.out_degree() if d==0)
  all_paths = partial(nx.all_simple_paths, G)
  ans = list(chaini(starmap(all_paths, product(roots, leaves))))
  return(ans)

In [73]:
get_subthreads(4445)

[[21680, 21681], [21680, 21682], [21680, 21683]]

In [71]:
def get_list_subthreads(threadID):
  conversations = []
  for i, val in enumerate(get_subthreads(threadID)):
    conversations.append(str(i)+":"+str(val)[1:-1].replace(',',''))
  return conversations

In [72]:
get_list_subthreads(4445)

['0:21680 21681', '1:21680 21682', '2:21680 21683']

In [9]:
def read_clusters(filename):
    # Read provided data
    clusters = {}
    cfile = ""
    all_points = set()
    for line in filename:
        if ':' in line:
            cfile = ':'.join(line.split(':')[:-1]).split('/')[-1]
            line = line.split(":")[-1]
        cluster = {int(v) for v in line.split()}
        clusters.setdefault(cfile, []).append(cluster)
        for v in cluster:
            all_points.add("{}:{}".format(cfile, v))
    return clusters, all_points

In [57]:
def create_contingency_table(gold, auto):
  table = []
  for i, v1 in gold.items():
    table_row = []
    for j, v2 in auto.items():
      table_row.append(len(v1.intersection(v2)))
    table.append(table_row)
  table = np.array(table)
  sum_rows = np.sum(table, axis = 1)
  sum_cols = np.sum(table, axis = 0)
  return table, sum_rows, sum_cols

In [74]:
gold = {'C1':{0,1,2},'C2':{0,1,3,4},'C3':{0,1,3,5}}

In [75]:
auto = {'CC1':{0,1,2},'CC2':{0,3,4,5}}

In [76]:
c,rows, cols = create_contingency_table(gold,auto)

In [77]:
c

array([[3, 1],
       [2, 3],
       [2, 3]])

In [10]:
def clusters_to_contingency(gold, auto):
    # A table, in the form of:
    # https://en.wikipedia.org/wiki/Rand_index#The_contingency_table
    table = {}
    for filename in auto:
        for i, acluster in enumerate(auto[filename]):
            aname = "auto.{}.{}".format(filename, i)
            current = {}
            table[aname] = current
            for j, gcluster in enumerate(gold[filename]):
                gname = "gold.{}.{}".format(filename, j)
                count = len(acluster.intersection(gcluster))
                if count > 0:
                    current[gname] = count
    counts_a = {}
    for filename in auto:
        for i, acluster in enumerate(auto[filename]):
            aname = "auto.{}.{}".format(filename, i)
            counts_a[aname] = len(acluster)
    counts_g = {}
    for filename in gold:
        for i, gcluster in enumerate(gold[filename]):
            gname = "gold.{}.{}".format(filename, i)
            counts_g[gname] = len(gcluster)

    
    return table, counts_a, counts_g

In [11]:
def variation_of_information(contingency, row_sums, col_sums):
    total = 0.0
    for row in row_sums:
        total += row_sums[row]

    H_UV = 0.0
    I_UV = 0.0
    for row in contingency:
        for col in contingency[row]:
            num = contingency[row][col]
            H_UV -= (num / total) * math.log(num / total, 2)
            I_UV += (num / total) * math.log(num * total / (row_sums[row] * col_sums[col]), 2)

    H_U = 0.0
    for row in row_sums:
        num = row_sums[row]
        H_U -= (num / total) * math.log(num / total, 2)
    H_V = 0.0
    for col in col_sums:
        num = col_sums[col]
        H_V -= (num / total) * math.log(num / total, 2)

    max_score = math.log(total, 2)
    VI = H_UV - I_UV
    
    scaled_VI = VI / max_score
    print("{:5.2f}   1 - Scaled VI".format(1 - scaled_VI))

###    # This version will spread values out more, but also introduces a
###    # dependence between the prediction and the scale factor, which feels
###    # odd (as mentioned in the paper that proposed it).
###    normalised_VI = 1 - (I_UV / H_UV)
###    print("1 - Normalised VI:", 1 - normalised_VI)

###    # http://jmlr.csail.mit.edu/papers/volume11/vinh10a/vinh10a.pdf
###    # Information Theoretic Measures for Clusterings Comparison: Variants,
###    # Properties, Normalization and Correction for Chance
###    # Vinh, et al. (2010), JMLR
###    #
###    #   NID = 1 - I(U, V) / max(H(U), H(V))
###    #
###   normalised_ID = 1 - (I_UV / max(H_U, H_V))
###    print("1 - Normalised ID:", 1 - normalised_ID)

In [12]:
def adjusted_rand_index(contingency, row_sums, col_sums, total_elements):
    # See https://en.wikipedia.org/wiki/Rand_index
    rand_index = 0.0
    total = float(total_elements)
    print('{:5.2f}   Total Elements'.format(total))
    for row in contingency:
        for col in contingency[row]:
            n = contingency[row][col]
            rand_index += n * (n-1) / 2.0
    sum_row_choose2 = 0.0
    for row in row_sums:
        n = row_sums[row]
        sum_row_choose2 += n * (n-1) / 2.0
    sum_col_choose2 = 0.0
    for col in col_sums:
        n = col_sums[col]
        sum_col_choose2 += n * (n-1) / 2.0
    print('{:5.2f}   Sum Row C 2'.format(sum_row_choose2))
    print('{:5.2f}   Sum Col C 2'.format(sum_col_choose2))
    random_index = (sum_row_choose2 * sum_col_choose2 * 2.0)/(total * (total - 1))
    print('{:5.2f}   Random Index'.format(random_index))
    max_index = 0.5 * (sum_row_choose2 + sum_col_choose2)
    print('{:5.2f}   Max Index'.format(max_index))
    adjusted_rand_index = (rand_index - random_index) / (max_index - random_index)

    print('{:5.2f}   Adjusted rand index'.format(adjusted_rand_index))

In [13]:
def adjusted_mutual_information(gold, auto):
    labels_true = []
    order = []
    cur = -1
    for filename in gold:
        for gcluster in gold[filename]:
            cur += 1
            for num in gcluster:
                order.append((filename, num))
                labels_true.append(cur)
    auto_map = {}
    cur = -1
    for filename in auto:
        for acluster in auto[filename]:
            cur += 1
            for num in acluster:
                auto_map[filename, num] = cur
    labels_pred = []
    for pair in order:
        labels_pred.append(auto_map[pair])

    score = metrics.adjusted_mutual_info_score(labels_true, labels_pred, 'max')
    print("{:5.2f}   adjusted mutual information".format(score))

In [14]:
def shen_f1(contingency, row_sums, col_sums, gold, auto):
    total = 0
    count = 0
    # For each gold cluster, calculate F-score relative to each auto cluster
    # and take the max. Then scale those scores by the size of the cluster.
    for col in col_sums:
        best = 0
        best_info = []
        col_sum = col_sums[col]
        count += col_sum
        for row in row_sums:
            n = 0
            if row in contingency and col in contingency[row]:
                n = contingency[row][col]
            if n > 0:
                row_sum = row_sums[row]
                p = n / row_sum
                r = n / col_sum
                f = 2 * p * r / (p + r)
                if f > best:
                    best = f
                    best_info = [n, row_sum, row]
        total += col_sum * best
    print("{:5.2f}   Shen F1".format(total / count))

In [15]:
def exact_match(gold, auto, skip_single=True):
    # P/R/F over complete clusters
    total_gold = 0
    total_matched = 0
    for filename in gold:
        for cluster in gold[filename]:
            if skip_single and len(cluster) == 1:
                continue
            total_gold += 1
            matched = False
            for ocluster in auto[filename]:
                if len(ocluster.symmetric_difference(cluster)) == 0:
                    matched = True
                    break
            if matched:
                total_matched += 1
    match = []
    subsets = []
    supersets = []
    other = []
    prefix = []
    suffix = []
    gap_free = []
    match_counts = []
    subsets_counts = []
    supersets_counts = []
    other_counts = []
    prefix_counts = []
    suffix_counts = []
    gap_free_counts = []
    total_auto = 0
    for filename in auto:
        for cluster in auto[filename]:
            if skip_single and len(cluster) == 1:
                continue
            total_auto += 1
            most_overlap = 0
            fraction = 0
            count = 0
            is_subset = False
            is_superset = False
            is_prefix = False
            is_suffix = False
            is_gap_free = False
            is_match = False
            for ocluster in gold[filename]:
                if len(ocluster.symmetric_difference(cluster)) == 0:
                    is_match = True
                    break

                overlap = len(ocluster.intersection(cluster))
                if overlap > most_overlap:
                    most_overlap = overlap
                    gaps = False
                    for v in ocluster:
                        if min(cluster) <= v <= max(cluster):
                            if v not in cluster:
                                gaps = True
                    fraction = 1 - (overlap / len(ocluster.union(cluster)))
                    count = len(ocluster.union(cluster)) - overlap

                    is_subset = (overlap == len(cluster))
                    is_superset = (overlap == len(ocluster))
                    if overlap == len(cluster) and (not gaps):
                        is_gap_free = True
                        if min(ocluster) == min(cluster):
                            is_prefix = True
                        if max(ocluster) == max(cluster):
                            is_suffix = True
            if is_match:
                match.append(fraction)
                match_counts.append(count)
            elif is_superset:
                supersets.append(fraction)
                supersets_counts.append(count)
            elif is_subset:
                subsets.append(fraction)
                subsets_counts.append(count)
                if is_prefix:
                    prefix.append(fraction)
                    prefix_counts.append(count)
                elif is_suffix:
                    suffix.append(fraction)
                    suffix_counts.append(count)
                elif is_gap_free:
                    gap_free.append(fraction)
                    gap_free_counts.append(count)
            else:
                other.append(fraction)
                other_counts.append(count)
    print("Property, Proportion, Av Frac, Av Count, Max Count, Min Count")
    if len(match) > 0:
        print("Match        {:5.2f} {:5.2f} {:5.2f}".format(len(match) / total_auto, sum(match) / len(match), sum(match_counts) / len(match)), max(match_counts), min(match_counts))
    if len(supersets) > 0:
        print("Super        {:5.2f} {:5.2f} {:5.2f}".format(len(supersets) / total_auto, sum(supersets) / len(supersets), sum(supersets_counts) / len(supersets)), max(supersets_counts), min(supersets_counts))
    if len(subsets) > 0:
        print("Sub          {:5.2f} {:5.2f} {:5.2f}".format(len(subsets) / total_auto, sum(subsets) / len(subsets), sum(subsets_counts) / len(subsets)), max(subsets_counts), min(subsets_counts))
    if len(prefix) > 0:
        print("Sub-Prefix   {:5.2f} {:5.2f} {:5.2f}".format(len(prefix) / total_auto, sum(prefix) / len(prefix), sum(prefix_counts) / len(prefix)))
    if len(suffix) > 0:
        print("Sub-Suffix   {:5.2f} {:5.2f} {:5.2f}".format(len(suffix) / total_auto, sum(suffix) / len(suffix), sum(suffix_counts) / len(suffix)))
    if len(gap_free) > 0:
        print("Sub-GapFree  {:5.2f} {:5.2f} {:5.2f}".format(len(gap_free) / total_auto, sum(gap_free) / len(gap_free), sum(gap_free_counts) / len(gap_free)))
    if len(other) > 0:
        print("Other        {:5.2f} {:5.2f} {:5.2f}".format(len(other) / total_auto, sum(other) / len(other), sum(other_counts) / len(other)))

    p, r, f = 0.0, 0.0, 0.0
    if total_auto > 0:
        p = 100 * total_matched / total_auto
    if total_gold > 0:
        r = 100 * total_matched / total_gold
    if total_matched > 0:
        f = 2 * p * r / (p + r)
    print("{:5.2f}   Matched clusters precision".format(p))
    print("{:5.2f}   Matched clusters recall".format(r))
    print("{:5.2f}   Matched clusters f-score".format(f))

In [16]:
def numbered_clusters(clusters):
    max_cluster = -1
    numbered = {}
    for cluster in clusters:
        max_cluster += 1
        for num in cluster:
            numbered[num] = max_cluster
    return numbered

In [17]:
def elsner_local_error(gold, auto, window=3):
    match = 0
    total = 0
    for filename in auto:
        anums = numbered_clusters(auto[filename])
        gnums = numbered_clusters(gold[filename])
        start = min(min(anums), min(gnums))
        end = max(max(anums), max(gnums)) + 1
        for num in range(start, end):
            if num in anums and num in gnums:
                for i in range(-window, 0):
                    pos = num + i
                    if pos in anums and pos in gnums:
                        if (gnums[pos] == gnums[num]) == (anums[pos] == anums[num]):
                            match += 1
                        total += 1
    print("{:5.2f}   Local-{}".format( match / total, window))

In [18]:
def one_to_one(contingency, row_sums, col_sums):
    row_to_num = {}
    col_to_num = {}
    num_to_row = []
    num_to_col = []
    for row_num, row in enumerate(row_sums):
        row_to_num[row] = row_num
        num_to_row.append(row)
    for col_num, col in enumerate(col_sums):
        col_to_num[col] = col_num
        num_to_col.append(col)

    min_cost_flow = pywrapgraph.SimpleMinCostFlow()
    start_nodes = []
    end_nodes = []
    capacities = []
    costs = []
    source = len(num_to_row) + len(num_to_col)
    sink = len(num_to_row) + len(num_to_col) + 1
    supplies = []
    tasks = min(len(num_to_row), len(num_to_col))
    for row, row_num in row_to_num.items():
        start_nodes.append(source)
        end_nodes.append(row_num)
        capacities.append(1)
        costs.append(0)
        supplies.append(0)
    for col, col_num in col_to_num.items():
        start_nodes.append(col_num + len(num_to_row))
        end_nodes.append(sink)
        capacities.append(1)
        costs.append(0)
        supplies.append(0)
    supplies.append(tasks)
    supplies.append(-tasks)
    for row, row_num in row_to_num.items():
        for col, col_num in col_to_num.items():
            cost = 0
            if col in contingency[row]:
                cost = - contingency[row][col]
            start_nodes.append(row_num)
            end_nodes.append(col_num + len(num_to_row))
            capacities.append(1)
            costs.append(cost)

    # Add each arc.
    for i in range(len(start_nodes)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                                    capacities[i], costs[i])
  
    # Add node supplies.
    for i in range(len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])

    # Find the minimum cost flow.
    min_cost_flow.Solve()

    # Score.
    total_count = sum(v for _, v in row_sums.items())
    overlap = 0
    for arc in range(min_cost_flow.NumArcs()):
        # Can ignore arcs leading out of source or into sink.
        if min_cost_flow.Tail(arc)!=source and min_cost_flow.Head(arc)!=sink:
            # Arcs in the solution have a flow value of 1. Their start and end nodes
            # give an assignment of worker to task.
            if min_cost_flow.Flow(arc) > 0:
                row_num = min_cost_flow.Tail(arc)
                col_num = min_cost_flow.Head(arc)
                col = num_to_col[col_num - len(num_to_row)]
                row = num_to_row[row_num]
                if col in contingency[row]:
                    overlap += contingency[row][col]
    print("{:5.2f}   one-to-one".format(overlap / total_count))

In [19]:
def correct_start(gold, auto, do_end=False):
    # P/R/F for identifying starting messages
    gold_starts = set()
    for filename in gold:
        for cluster in gold[filename]:
            start = min(cluster)
            if do_end:
                start = max(cluster)
            gold_starts.add((filename, start))

    auto_starts = set()
    for filename in auto:
        for cluster in auto[filename]:
            start = min(cluster)
            if do_end:
                start = max(cluster)
            auto_starts.add((filename, start))

    match = len(gold_starts.intersection(auto_starts))

    p = 100 * match / len(auto_starts)
    r = 100 * match / len(gold_starts)
    f = 0.0
    if match > 0:
        f = 2 * p * r / (p + r)
    prefix = "End" if do_end else "Start"
    print("{:5.2f}   {} Precision".format(p, prefix))
    print("{:5.2f}   {} Recall".format(r, prefix))
    print("{:5.2f}   {} F-score".format(f, prefix))

In [20]:
def start_and_end_together(gold, auto):
    # P/R/F for identifying starting messages
    total = 0
    together = 0
    for filename in gold:
        for cluster in gold[filename]:
            total += 1
            start = min(cluster)
            end = max(cluster)
            found = False
            for ocluster in auto[filename]:
                if start in ocluster and end in ocluster:
                    found = True
                    break
            if found:
                together += 1
    print("{:5.2f}   Start and end of cluster still together".format( together / total))

In [21]:
pip install ortools

In [22]:
import argparse
import math
import sys

from ortools.graph import pywrapgraph
from sklearn import metrics

In [23]:
gt = get_clusters(1)
pred = get_clusters(1)

In [25]:
gt = ['0:0 1 2',
 '1:0 1 3 4',
 '2:0 1 3 5']

In [26]:
pred = ['0:0 1 2 3 4 5']

In [27]:
gold, gpoints = read_clusters(gt)
auto, apoints = read_clusters(pred)

In [28]:
gold

{'0': [{0, 1, 2}], '1': [{0, 1, 3, 4}], '2': [{0, 1, 3, 5}]}

In [29]:
gpoints

{'0:0', '0:1', '0:2', '1:0', '1:1', '1:3', '1:4', '2:0', '2:1', '2:3', '2:5'}

In [30]:
auto

{'0': [{0, 1, 2, 3, 4, 5}]}

In [31]:
apoints

{'0:0', '0:1', '0:2', '0:3', '0:4', '0:5'}

In [32]:
def get_total_points(gpoints):
  points = []
  for i in gpoints:
    x,y = i.split(':')
    points.append(y)
  total_points = len(set(points))
  return(total_points)

In [33]:
get_total_points(gpoints)

6

In [34]:
import re, string
import sys, os
import itertools

#############
# Calculates the Omega Index as described in Collins and Dent 1988
#############
def omega_index(gold, auto):
  # return dictionary associating objects w/ clusters
  def get_post_clusters_dict(data):
    val_key_dict = {}
    for key,val in data.items():
      for i in val[0]:
        if i in val_key_dict:
          j = val_key_dict.get(i)
          val_key_dict[i] = j + [key]
        else:
          val_key_dict[i] = [key]
    for key,val in val_key_dict.items():
      val_key_dict[key] = '#'.join(val)
    return val_key_dict


  # cross-tabulation of # times each pair of objects is clustered together by each solution
  # takes two dictionaries associating objects with clusters (two solutions)
  # dictionaries have identical keys (objects)
  # returns dictionary w/ info on pair clustering for the two solutions
  def table(dict1, dict2):

      objects = dict1.keys()
      pairs = itertools.combinations(objects, 2) # every pair of objects

      tabledict = {}

      for p in pairs:

          sol1w1 = dict1[p[0]]
          sol1w2 = dict1[p[1]]
          # number of clusters in which pair is together (solution 1)
          tog1 = len(set(sol1w1) & set(sol1w2))

          sol2w1 = dict2[p[0]]
          sol2w2 = dict2[p[1]]
          # number of clusters in which pair is together (solution 2)
          tog2 = len(set(sol2w1) & set(sol2w2))

          #if sol1w1 == ['0'] or sol1w2 == ['0'] or sol2w1 == ['0'] or sol2w2 == ['0']:
          #    continue
          #else:
          tabledict[p] = (tog1, tog2)
      return tabledict
      
  # calculate marginals from the table
  # solNumber is the number of the solution (0 or 1)
  # returns dictionary of marginals for that solution
  def margins(table, solNumber):
      tv = table.values()
      marginals = {}
      d1vals = [e[solNumber] for e in tv]
      for k in list(set(d1vals)):
          marginals[k] = d1vals.count(k)
      return marginals

  # uses cross-tabulation to calculate omega index
  # returns omega index
  def omega(tab):
      sol1 = margins(tab, 0)
      sol2 = margins(tab, 1)

      maxj = min([max(sol1.keys()), max(sol2.keys())])

      agree = 0
      for pair in tab:
          entry = tab[pair]
          if entry[0] == entry[1]:
              agree += 1
      observed = agree * len(tab)
      
      count = -1
      expected = 0
      while count < maxj:
          count += 1
          c1 = sol1[count]
          c2 = sol2[count]
          expected += (c1 * c2)
      num = observed - expected
      den = (len(tab)**2) - expected
      return float(num) / float(den)

  d1 = get_post_clusters_dict(gold)
  d2 = get_post_clusters_dict(auto)
  tab = table(d1, d2)
  score = omega(tab)
  return score

In [35]:
contingency, row_sums, col_sums = None, None, None
contingency, row_sums, col_sums = clusters_to_contingency(gold, auto)
print('Variation of Information')
variation_of_information(contingency, row_sums, col_sums)
print('Shen F1')
shen_f1(contingency, row_sums, col_sums, gold, auto)
print('Omega Index')
omega_index(gold, auto)

Variation of Information
 0.81   1 - Scaled VI
Shen F1
 0.18   Shen F1
Omega Index


KeyError: ignored